Setup the environment

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyodbc
import warnings

warnings.filterwarnings("ignore")

# Set the environment
np.random.seed(1234)
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

Connect to MS SQL Server

In [ ]:
# Set the connection string to SQL Server
trst_conn= "Yes"
driver = "ODBC Driver 17 for SQL Server"
server = "localhost"
src_database = "SharkAttacks"     # Source DB
# dest_database = "Northwind2012STG"    # Destination DB

In [ ]:
# Create a new connection to source DB (Trusted Connection -> Windows Authentication)
trusted_conn_src = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={src_database};TRUSTED_CONNECTION={trst_conn}')

Extract data from MS SQL Server

In [ ]:
# Load table from source DB into a dataframe
read_query='''
SELECT *  FROM tblSharkAttacks
'''
df = pd.read_sql(read_query, trusted_conn_src)

Data Exploration

In [ ]:
df

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
# Show summary statistics
df.describe(include = 'all')

Data Transformation

In [ ]:
# Remove empty rows/columns
df.dropna(how="all", axis=0, inplace=True) # Rows
df.dropna(how="all", axis=1, inplace=True) # Columns
df.info()

In [ ]:
# Remove duplicate rows
df.drop_duplicates(keep='first', inplace=True)
df.shape

In [ ]:
# Remove leading and trailing spaces from 'object' (string) columns
for col in df.select_dtypes(['object']).columns:
    df[col] = df[col].str.strip()

In [ ]:
# Remove unwanted characters (<, %, ?) from all rows
df = df.replace(r'[<%?]', '', regex=True)

In [ ]:
# Replace a character with another character in a specific string column
df['Date'] = df['Date'].str.replace('Reported ','')
df['Time'] = df['Time'].str.replace('h',':')
df

In [ ]:
# Rename the columns Fatal (Y/N) to Fatal and Investigator or Source to Source
df = df.rename(columns={'Fatal (Y/N)':'Fatal', 'Investigator or Source':'Source'})
df.head()

In [ ]:
# Drop unwanted columns
df.drop(['Year', 'pdf', 'href formula', 'href', 'Case Number1', 'Case Number2', 'original order', 'Source'], axis='columns', inplace=True)

In [ ]:
df.info()

In [ ]:
df.nunique()

In [ ]:
# Check for nulls
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df

In [ ]:
# Drop rows with empty values in a column
df.dropna(subset=['Case Number'], inplace=True)

In [ ]:
df.info()

In [ ]:
# Replace empty values for 'object' types with the most used value
string_columns = df.select_dtypes(include=['object']).columns
df[string_columns] = df[string_columns].fillna(df[string_columns].mode().iloc[0])

In [ ]:
df.info()

In [ ]:
# Replace numerical nulls with mean
df.fillna(df.mean(numeric_only=True).round(1), inplace=True)

In [ ]:
# Convert data types
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Drop rows with empty values in a date column
df.dropna(subset=['Date'], inplace=True)

In [ ]:
# Extract date parts into new columns
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Month Name'] = df['Date'].dt.strftime('%b')         # Extract Month date part in abbreviated format (e.g. August -> Aug)
# df['Month Name'] = df['Date'].dt.month_name()
df['Day'] = df['Date'].dt.day
df['Weekday (Name)'] = df['Date'].dt.strftime('%a')     # Extract Weekday name date part in abbreviated format (e.g. Tuesday -> Tue)
# df['Weekday (Name)'] = df['Date'].dt.day_name()
df['Decade'] = df['Year'] - df['Year'] % 10

In [ ]:
df

In [ ]:
df.info()

In [ ]:
# Create buckets (bins) for Age
# Define the bins and labels
bins = [0, 12, 19, 35, 55, float('inf')]
labels = ['0-11', '12-18', '19-34', '35-54', '55+']

# Create a new column for age groups
df['Age Group'] = pd.cut(df['Age'], bins=bins, labels=labels, right=False)
df

In [ ]:
# Split the Location column and create a new column with the first part of the splitted column
df['Location_1'] = df['Location'].str.split(',').str[0]

# Split the Location column and create a new column with the second part of the splitted column
df['County'] = df['Location'].str.split(',').str[1]
df.head()

In [ ]:
# Remove the original column
df.drop(['Location'], axis='columns', inplace=True)
df.head()

In [ ]:
# Rename the new column from the split, back to the name of the original column
df = df.rename(columns={'Location_1':'Location'})
df.head()

Save transformed data to flat file (CSV)

In [ ]:
# Save cleaned dataframe to .CSV
df.to_csv('sharkattacks_cleaned.csv', index=False)